In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

GPU is available. Using GPU: Tesla T4


In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


path = "/content/drive/MyDrive/Data_Image"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

Image_dataset = datasets.ImageFolder(root=path, transform=transform)


In [ ]:
# Chia dữ liệu thành tập train và tập test
train_dataset, test_dataset = random_split(Image_dataset, [0.7, 0.3])

batch_size = 32
# Tạo DataLoader để nạp dữ liệu theo batch
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
len(Image_dataset)

1346

In [ ]:
import torch.nn as nn
from torchvision.models import vision_transformer

class VisionTransformerModel(nn.Module):
    def __init__(self, num_classes=16):
        super(VisionTransformerModel, self).__init__()

        self.vit = vision_transformer.vit_b_16(weights=None).to(device)

        n_inputs = self.vit.heads[0].in_features
        self.vit.heads[0] = nn.Sequential(
            nn.Linear(n_inputs, num_classes),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.vit(x)
        return x

num_classes = len(Image_dataset.classes)
vision_transformer_model = VisionTransformerModel(num_classes).to(device)

In [ ]:
print(vision_transformer_model)

VisionTransformerModel(
  (vit): VisionTransformer(
    (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (encoder): Encoder(
      (dropout): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (encoder_layer_0): EncoderBlock(
          (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attention): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (0): Linear(in_features=768, out_features=3072, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
            (3): Linear(in_features=3072, out_features=768, bias=True)
            (4): Dropout(p=0.0, inplace=False)
          )
        )
        (encoder_layer_1): EncoderBlock(
          (l

In [ ]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn):
    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(data_loader):
        # Chuyển dữ liệu và mô hình lên GPU
        X, y = X.to(device), y.to(device)
        model.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn):
    test_loss, test_acc = 0, 0
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode():
        for (X, y) in data_loader:
            # Chuyển dữ liệu lên GPU
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y).item()
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )

        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(vision_transformer_model.parameters(), lr=0.0001)

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()  # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc


In [ ]:
torch.manual_seed(42)

epochs = 30
for epoch in range(epochs):
    print(f"Epoch: {epoch}\n---------")
    vision_transformer_model.train()

    train_step(data_loader=train_loader,
        model=vision_transformer_model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )

    vision_transformer_model.eval()
    test_step(data_loader=test_loader,
        model=vision_transformer_model,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )

Epoch: 0
---------
Train loss: 2.98665 | Train accuracy: 11.60%
Test loss: 2.71946 | Test accuracy: 10.11%

Epoch: 1
---------
Train loss: 2.67470 | Train accuracy: 14.43%
Test loss: 2.65083 | Test accuracy: 15.47%

Epoch: 2
---------
Train loss: 2.55144 | Train accuracy: 18.48%
Test loss: 2.46888 | Test accuracy: 15.71%

Epoch: 3
---------
Train loss: 2.47683 | Train accuracy: 19.22%
Test loss: 2.45511 | Test accuracy: 19.26%

Epoch: 4
---------
Train loss: 2.42880 | Train accuracy: 20.53%
Test loss: 2.31390 | Test accuracy: 26.80%

Epoch: 5
---------
Train loss: 2.30177 | Train accuracy: 23.40%
Test loss: 2.15339 | Test accuracy: 23.41%

Epoch: 6
---------
Train loss: 2.15155 | Train accuracy: 28.70%
Test loss: 2.16862 | Test accuracy: 20.69%

Epoch: 7
---------
Train loss: 2.11554 | Train accuracy: 28.26%
Test loss: 2.35116 | Test accuracy: 22.77%

Epoch: 8
---------
Train loss: 1.93808 | Train accuracy: 35.94%
Test loss: 1.97895 | Test accuracy: 34.40%

Epoch: 9
---------
Train los